In [1]:
from sqlalchemy import create_engine, MetaData, Table
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy.orm import Session, sessionmaker
from sqlalchemy import Column, Integer, String, BigInteger, VARCHAR, SmallInteger
from sqlalchemy import *
import requests
import csv
import os
from requests.auth import HTTPBasicAuth
from config import SQALCHEMY_DATABASE_URL
import datetime 

class DictMixIn():
    def to_dict(self):
        return {
            column.name: getattr(self, column.name)
            if not isinstance(getattr(self, column.name), datetime.datetime)
            else getattr(self, column.name).isoformat()
            for column in self.__table__.columns
        }



Base = declarative_base()
engine = create_engine(
    SQALCHEMY_DATABASE_URL, 
    pool_recycle=3600,
    pool_pre_ping=True
)
# Base.prepare(engine, reflect=True)
# Session = sessionmaker(bind=engine)
session = Session(engine)


class Product_nutrients(Base, DictMixIn):
    __tablename__ = "product_nutrients"
    product_id = Column(BigInteger, primary_key=True)
    product_name = Column(VARCHAR (120))
    price = Column(Integer)
    aisle_id = Column(Integer)
    department_id = Column(Integer)
    text_API = Column(Integer)
    ENERC_KCAL = Column(Integer)
    FAT = Column(Integer)
    CHOCDF = Column(Integer)
    FIBTG = Column(Integer)
    PROCNT = Column(Integer)
    
class Products(Base, DictMixIn):
    __tablename__ = "products"
    product_id = Column(BigInteger, primary_key=True)
    product_name = Column(VARCHAR (120))
    price = Column(Integer)
    aisle_id = Column(Integer)
    department_id = Column(Integer)


C:\Users\User\.virtualenvs\inventory-vGz33zy2\lib\site-packages\flask_sqlalchemy\__init__.py:814: UserWarning: Neither SQLALCHEMY_DATABASE_URI nor SQLALCHEMY_BINDS is set. Defaulting SQLALCHEMY_DATABASE_URI to "sqlite:///:memory:".
  'Neither SQLALCHEMY_DATABASE_URI nor SQLALCHEMY_BINDS is set. '
C:\Users\User\.virtualenvs\inventory-vGz33zy2\lib\site-packages\flask_sqlalchemy\__init__.py:835: FSADeprecationWarning: SQLALCHEMY_TRACK_MODIFICATIONS adds significant overhead and will be disabled by default in the future.  Set it to True or False to suppress this warning.
  'SQLALCHEMY_TRACK_MODIFICATIONS adds significant overhead and '


In [2]:
product_query = session.query(Products).all()


In [3]:
dict_list = [p.to_dict() for p in product_query]

In [ ]:
filename ='../Resources/product_nutrients.csv'

url = "https://api.edamam.com/api/food-database/parser"
data = []
for d in dict_list[5776:]:

    querystring = {'ingr': d['product_name'],
               'app_id': 'a67c4d31',
               'app_key': '82c3ff594705e85d5561caf56cfcc3a1'
              }
    response = requests.get(url, params=querystring).json()
    
    if len(response['parsed']) > 0:
        
        r_dict  = {
        'text_API': response['parsed'][0]['food']['label']
        }
        
        try:
            r_dict.update({'ENERC_KCAL': response['parsed'][0]['food']['nutrients']['ENERC_KCAL']})
        except:
            r_dict.update({'ENERC_KCAL' : 0}) 
        
        try:
            r_dict.update({'FAT': response['parsed'][0]['food']['nutrients']['FAT']})
        except:
            r_dict.update({'FAT' : 0})
                          
        try:
            r_dict.update({'CHOCDF': response['parsed'][0]['food']['nutrients']['CHOCDF']})
        except:
            r_dict.update({'CHOCDF' : 0})        
        try:
            r_dict.update({'FIBTG': response['parsed'][0]['food']['nutrients']['FIBTG']})
        except:
            r_dict.update({'FIBTG' : 0})        
        try:
            r_dict.update({'PROCNT': response['parsed'][0]['food']['nutrients']['PROCNT']})
        except:
            r_dict.update({'PROCNT': 0} )      


        full_data = d
    
        full_data.update(r_dict)
        data.append(full_data)
        

    else:
        full_data = d
        full_data.update({
            'text_API': None,
            'ENERC_KCAL': None,
            'FAT': None,
            'CHOCDF': None,
            'FIBTG': None,
            'PROCNT': None
        })
        data.append(full_data)

    file_exists = os.path.isfile(filename)


    
    # writing to csv file  
    with open(filename, 'a', encoding='utf-8') as csvfile: 
        # creating a csv writer object
        fieldnames = full_data.keys()
        writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
        
        if not file_exists:
            writer.writeheader()

        # writing the data rows  
        writer.writerow(full_data)
    
        

    
    
    